# Lab4-Assignment about Named Entity Recognition and Classification

This notebook describes the assignment of Lab 4 of the text mining course. We assume you have succesfully completed Lab1, Lab2 and Lab3 as welll. Especially Lab2 is important for completing this assignment.

**Learning goals**
* going from linguistic input format to representing it in a feature space
* working with pretrained word embeddings
* train a supervised classifier (SVM)
* evaluate a supervised classifier (SVM)
* learn how to interpret the system output and the evaluation results
* be able to propose future improvements based on the observed results


## Credits
This notebook was originally created by [Marten Postma](https://martenpostma.github.io) and [Filip Ilievski](http://ilievski.nl) and adapted by Piek vossen

## [Points: 18] Exercise 1 (NERC): Training and evaluating an SVM using CoNLL-2003

**[4 point] a) Load the CoNLL-2003 training data using the *ConllCorpusReader* and create for both *train.txt* and *test.txt*:**

    [2 points]  -a list of dictionaries representing the features for each training instances, e..g,
    ```
    [
    {'words': 'EU', 'pos': 'NNP'}, 
    {'words': 'rejects', 'pos': 'VBZ'},
    ...
    ]
    ```

    [2 points] -the NERC labels associated with each training instance, e.g.,
    dictionaries, e.g.,
    ```
    [
    'B-ORG', 
    'O',
    ....
    ]
    ```

In [7]:
from nltk.corpus.reader import ConllCorpusReader
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('C:/Users/lisam/OneDrive/Year 3/Period 4/Text Mining/ba-text-mining-gr43/lab_sessions/lab4/CONLL2003/CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

training_features = []
training_gold_labels = []

for token, pos, ne_label in train.iob_words():
    a_dict = {'words':token,'pos':pos}
    training_features.append(a_dict)
    training_gold_labels.append(ne_label)
   

In [8]:
### Adapt the path to point to the CONLL2003 folder on your local machine
train = ConllCorpusReader('C:/Users/lisam/OneDrive/Year 3/Period 4/Text Mining/ba-text-mining-gr43/lab_sessions/lab4/CONLL2003/CONLL2003', 'test.txt', ['words', 'pos', 'ignore', 'chunk'])
test_features = []
test_gold_labels = []
for token, pos, ne_label in train.iob_words():
    a_dict = {'words':token,'pos':pos}
    test_features.append(a_dict)
    test_gold_labels.append(ne_label)

**[2 points] b) provide descriptive statistics about the training and test data:**
* How many instances are in train and test?
* Provide a frequency distribution of the NERC labels, i.e., how many times does each NERC label occur?
* Discuss to what extent the training and test data is balanced (equal amount of instances for each NERC label) and to what extent the training and test data differ?

Tip: you can use the following `Counter` functionality to generate frequency list of a list:

In [9]:
from collections import Counter 

my_list=[1,2,1,3,2,5]
Counter(my_list)


Counter({1: 2, 2: 2, 3: 1, 5: 1})

In [17]:
from collections import Counter 

path = 'C:/Users/lisam/OneDrive/Year 3/Period 4/Text Mining/ba-text-mining-gr43/lab_sessions/lab4/CONLL2003/CONLL2003'

train = ConllCorpusReader(path, 'train.txt', ['words', 'pos', 'ignore', 'chunk'])
test = ConllCorpusReader(path, 'test.txt', ['words', 'pos', 'ignore', 'chunk'])

def count_labels(data):
    
    labels = []
    for token, pos, ne_label in data.iob_words():
        labels.append(ne_label)
    return labels, Counter(labels)

def calculate_percentages(label_counts, total):
    print("\nDstribution of NERC labels (in percentage):")
    for label, count in label_counts.items():
        percentage = (count / total) * 100
        print(f"{label}: {percentage:.2f}%")

def get_statistics(data, name):
    print(f"\n{name}:")
    
    # Get label counts
    labels, label_counts = count_labels(data)
    num_instances = len(labels)

    print(f"Number of instances: {num_instances}")
    
    print("\nFrequency distribution of NERC labels:")
    for label, count in label_counts.items():
        print(f"{label}: {count}")

    calculate_percentages(label_counts, num_instances)

get_statistics(train, "Train")
get_statistics(test, "Test")


Train:
Number of instances: 203621

Frequency distribution of NERC labels:
B-ORG: 6321
O: 169578
B-MISC: 3438
B-PER: 6600
I-PER: 4528
B-LOC: 7140
I-ORG: 3704
I-MISC: 1155
I-LOC: 1157

Dstribution of NERC labels (in percentage):
B-ORG: 3.10%
O: 83.28%
B-MISC: 1.69%
B-PER: 3.24%
I-PER: 2.22%
B-LOC: 3.51%
I-ORG: 1.82%
I-MISC: 0.57%
I-LOC: 0.57%

Test:
Number of instances: 46435

Frequency distribution of NERC labels:
O: 38323
B-LOC: 1668
B-PER: 1617
I-PER: 1156
I-LOC: 257
B-MISC: 702
I-MISC: 216
B-ORG: 1661
I-ORG: 835

Dstribution of NERC labels (in percentage):
O: 82.53%
B-LOC: 3.59%
B-PER: 3.48%
I-PER: 2.49%
I-LOC: 0.55%
B-MISC: 1.51%
I-MISC: 0.47%
B-ORG: 3.58%
I-ORG: 1.80%


Answers:

- There are 203621 instances of training data, and 46435 of testing, as it can be seen from the output of the cell below.
- The frequency distibution can also be seen in the output of the cell below, which for training data is:
B-ORG: 6321

O: 169578

B-MISC: 3438

B-PER: 6600

I-PER: 4528

B-LOC: 7140

I-ORG: 3704

I-MISC: 1155

I-LOC: 1157

and for test data is:

O: 38323

B-LOC: 1668

B-PER: 1617

I-PER: 1156

I-LOC: 257

B-MISC: 702

I-MISC: 216

B-ORG: 1661

I-ORG: 835

- The part with percentage count in the output of the cell below clearly indicates the imbalance of training and test data when looking at them separately. For both test and train data, the "O" label, which is used to label the words that are not a part of any category of named entity, dominates with 83.28% and 82.53%, respectively. This can lead to biases in the model and incorrect labeling. And looking at percentage of labels that start with I or B and comparing them, we can see that in general, NERC labels that start with B ahave higher percentages, which means that the most named entities are shorter rather than longer. If comparing test and training data with each other, they do have similar percentages for the same labels, not having differences of more than ~1%, some of them having a difference only of 0.02% (for example I-LOC, with 0.57% and 0.55%).

**[2 points] c) Concatenate the train and test features (the list of dictionaries) into one list. Load it using the *DictVectorizer*. Afterwards, split it back to training and test.**

Tip: You’ve concatenated train and test into one list and then you’ve applied the DictVectorizer.
The order of the rows is maintained. You can hence use an index (number of training instances) to split the_array back into train and test. Do NOT use: `
from sklearn.model_selection import train_test_split` here.


In [13]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
vec = DictVectorizer()
the_array = # your code here

**[4 points] d) Train the SVM using the train features and labels and evaluate on the test data. Provide a classification report (sklearn.metrics.classification_report).**
The train (*lin_clf.fit*) might take a while. On my computer, it took 1min 53s, which is acceptable. Training models normally takes much longer. If it takes more than 5 minutes, you can use a subset for training. Describe the results:
* Which NERC labels does the classifier perform well on? Why do you think this is the case?
* Which NERC labels does the classifier perform poorly on? Why do you think this is the case?

In [8]:
from sklearn import svm

In [9]:
lin_clf = svm.LinearSVC()

In [3]:
##### [ YOUR CODE SHOULD GO HERE ]
# lin_clf.fit( # your code here

**[6 points] e) Train a model that uses the embeddings of these words as inputs. Test again on the same data as in 2d. Generate a classification report and compare the results with the classifier you built in 2d.**

In [10]:
# your code here

## [Points: 10] Exercise 2 (NERC): feature inspection using the [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)
**[6 points] a. Perform the same steps as in the previous exercise. Make sure you end up for both the training part (*df_train*) and the test part (*df_test*) with:**
* the features representation using **DictVectorizer**
* the NERC labels in a list

Please note that this is the same setup as in the previous exercise:
* load both train and test using:
    * list of dictionaries for features
    * list of NERC labels
* combine train and test features in a list and represent them using one hot encoding
* train using the training features and NERC labels

In [ ]:
import pandas

In [ ]:
##### Adapt the path to point to your local copy of NERC_datasets
path = '/Users/piek/Desktop/ONDERWIJS/data/nerc_datasets/kaggle/ner_v2.csv'
kaggle_dataset = pandas.read_csv(path, error_bad_lines=False)

In [ ]:
len(kaggle_dataset)

In [ ]:
df_train = kaggle_dataset[:100000]
df_test = kaggle_dataset[100000:120000]
print(len(df_train), len(df_test))

**[4 points] b. Train and evaluate the model and provide the classification report:**
* use the SVM to predict NERC labels on the test data
* evaluate the performance of the SVM on the test data

Analyze the performance per NERC label.

## End of this notebook